### Compare new P4 tariffs

In [16]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
file_p4_new_tariffs = 'P4_20190410.csv'
file_partner_new_tariffs = 'Partner_1_1_2019.csv'
file_allowed_list = 'DMI.dmi_allowed_list_export_Tue_Apr_09_2019.csv'

In [72]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)

sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=2,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=2,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=2,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=2,DATA_RATE,DATA_RATE*1.2) DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=267
AND end_date IS NULL
'''

df_ocs_partner = pd.read_sql_query(sql_srt, cnxn)
print('Partner cost:')
display(df_ocs_partner.head(3))

sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=2,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=2,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=2,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=2,DATA_RATE,DATA_RATE*1.2) DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=268
AND end_date IS NULL AND end_date
'''

sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=2,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=2,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=2,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=2,DATA_RATE,DATA_RATE*1.2) DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=268
AND (end_date IS NULL OR end_date = '2019-04-10 00:00:00')
AND start_date <> '2019-04-10 00:00:00'
'''

df_ocs_p4 = pd.read_sql_query(sql_srt, cnxn)
print('\nP4 cost:')
df_ocs_p4 = df_ocs_p4
display(df_ocs_p4.head(3))

cnxn.close()

Partner cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,1.188104,1.188104,0.299547,12.285824,59.0,1.0,False
1,2.626872,2.626872,0.540158,17.199903,95.0,1.0,False
2,0.792080,0.000000,0.359676,11.997587,108.0,1.0,False



P4 cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,3.1824,0.0,0.3384,1.872,171.0,2.0,False
1,2.7460,0.0,0.5260,11.612,95.0,2.0,False
2,1.5960,0.0,0.2830,8.294,1100.0,2.0,False


In [73]:
# Загружаем новые тарифы Partner и P4 и Allowed list

df_partner = pd.read_csv(join(load_folder, file_partner_new_tariffs),sep=';')
print('Тарифы Partner:')
display(df_partner.head(3))

df_p4 = pd.read_csv(join(load_folder, file_p4_new_tariffs),sep=';')

df_p4.Data_Increment_KB.replace({'1 kB':1, '10 kB':10, '1kB':1}, inplace=True)
df_p4.Voice_Increment_SEC.replace({'30/1':30}, inplace=True)
df_p4.MTC.fillna(0, inplace=True)

print('Тарифы P4:')
display(df_p4.head(3))

df_al=pd.read_csv(join(load_folder, file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al, df_tadig[['TADIG_CODE','NETWORK_ID']], how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE
0,Albania,TELEKOM ALBANIA SH.A,ALBAM,0.449,0.0,0.052,0.040,1,1,yes,yes
1,Albania,ALBtelecom sh.a.,ALBEM,0.390,0.0,0.040,0.039,60,1,yes,yes
2,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,0.0,0.080,0.080,1,1,yes,no


In [74]:
#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','P4_CAMEL','P4_LTE',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
#print('Тарифы Partner и P4:')
#display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
#print('Тарифы Partner и P4 с TADIG:')
#display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
#print('Тарифы Partner и P4 с привязкой к Allowed list:')
#display(df_s1_s2_sponsor.head())

# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR','TADIG_CODE_AL','NETWORK_ID','P4_CAMEL','P4_LTE'
         ,'Voice_Increment_SEC','Data_Increment_KB']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
#df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head(3)

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,INDEU,40456.0,IdeaUP IN,India,IDEA,0.1,0.0,0.06,0.02,NaN,NaN,NaN,NaN
87,INDIB,40570.0,IDEA Bih IN,India,IDEA,0.1,0.0,0.06,0.02,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
146,Mozambique,MOVITEL S.A,MOZVG,0.7,0.0,0.07,0.042,60,10,yes,no,NaN,NaN,NaN,NaN



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,P4_CAMEL,P4_LTE,Voice_Increment_SEC,Data_Increment_KB
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,NaN,NaN,NaN,NaN,S1,ALBVF,12.0,NaN,NaN,NaN,NaN
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,0.8,0.0,0.08,0.08,S1,AIACW,20.0,yes,no,1,1.0


In [75]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

print('Partner:')
df = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)

print('\nP4:')
df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df[df.values > 1]
display(s2)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

Partner:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



P4:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,
412.0,2,1,1,1


In [76]:
# Проверить изменения тарифов

df=pd.merge(df_s1_s2_sponsor, df_ocs_p4, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG P4:")
cols_p4 = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'P4_LTE','P4_CAMEL','is_discounted','NETWORK_ID']
display(df.loc[((df.MB_S2.notnull()) & (df.SPONSOR_ID.isnull())), cols_p4])

# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG Partner:")
s1 = df[(df.MB_S1.isnull()) & (df.SPONSOR_ID.notnull()) & (df.is_discounted == True)].NETWORK_ID.values
display(df_tadig[df_tadig.NETWORK_ID.isin(s1)])

print("\nПропавшие TADIG P4:")
s2 = df[(df.MB_S2.isnull()) & (df.SPONSOR_ID.notnull()) & (df.is_discounted == True)].NETWORK_ID.values
display(df_tadig[df_tadig.NETWORK_ID.isin(s2)])

Новые TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,P4_LTE,P4_CAMEL,is_discounted,NETWORK_ID
24,Bosnia and Herzegov,BH Telecom JSC Sarajevo,BIHPT,0.520,0.0,0.075,0.014,no,yes,NaN,102.0
102,NaN,NaN,NaN,0.700,0.0,0.070,0.042,no,yes,NaN,NaN
204,Montenegro,Telenor d o o Podgorica,MNEPM,0.600,0.0,0.035,0.019,yes,yes,NaN,623.0
205,Montenegro,Telenor d o o Podgorica,YUGPM,0.600,0.0,0.035,0.019,yes,yes,NaN,623.0
210,Myanmar (Burma),Telenor Myanmar Limited,MMRTN,0.600,0.0,0.035,0.019,no,no,NaN,633.0
273,Thailand,dtac TriNet Co Ltd,THADT,0.600,0.0,0.035,0.019,yes,yes,NaN,956.0
296,Albania,TELEKOM ALBANIA SH A,ALBAM,0.449,0.0,0.052,0.040,yes,yes,NaN,9.0
300,Bangladesh,Grameenphone Ltd,BGDGP,0.600,0.0,0.035,0.019,no,no,NaN,66.0
311,Congo Dem. Rep,Africell,CODAC,0.700,0.0,0.140,4.200,no,no,NaN,208.0
312,Costa Rica,Instituto Costarricense de Electricidad ICE,CRICR,0.400,0.0,0.150,0.060,no,yes,NaN,221.0



Пропавшие TADIG Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
6,Albania,Eagle Mobile,ALBEM,10.0
47,Bahamas,Bahamas Telco. Comp.,BHSBH,59.0
51,Bangladesh,Airtel/Warid,BGDWT,69.0
54,Bangladesh,Robi/Aktel,BGDAK,64.0
57,Belarus,BeST,BLRBT,75.0
58,Belarus,JLLC Mobile TeleSystems,BLR02,77.0
60,Belgium,ORANGE Belgium nv/SA,BELMO,81.0
77,Bosnia and Herzegov,Public Enterprise Croatian Telecom JSC,BIHER,103.0
78,Bosnia and Herzegov,RS Telecommunications JSC Banja Luka,BIHMS,104.0
121,Canada,Bragg Communications Inc,CANEL,1163.0



Пропавшие TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
78,Bosnia and Herzegov,RS Telecommunications JSC Banja Luka,BIHMS,104.0


In [77]:
print("Новые TADIG P4:")
cols_p4 = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'P4_LTE','P4_CAMEL']
display(df.loc[((df.MB_S2.notnull()) & (df.SPONSOR_ID.isnull()) & (df.TADIG_CODE.notnull())), cols_p4])

Новые TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,P4_LTE,P4_CAMEL
24,Bosnia and Herzegov,BH Telecom JSC Sarajevo,BIHPT,0.520,0.0,0.075,0.014,no,yes
204,Montenegro,Telenor d o o Podgorica,MNEPM,0.600,0.0,0.035,0.019,yes,yes
205,Montenegro,Telenor d o o Podgorica,YUGPM,0.600,0.0,0.035,0.019,yes,yes
210,Myanmar (Burma),Telenor Myanmar Limited,MMRTN,0.600,0.0,0.035,0.019,no,no
273,Thailand,dtac TriNet Co Ltd,THADT,0.600,0.0,0.035,0.019,yes,yes
296,Albania,TELEKOM ALBANIA SH A,ALBAM,0.449,0.0,0.052,0.040,yes,yes
300,Bangladesh,Grameenphone Ltd,BGDGP,0.600,0.0,0.035,0.019,no,no
311,Congo Dem. Rep,Africell,CODAC,0.700,0.0,0.140,4.200,no,no
312,Costa Rica,Instituto Costarricense de Electricidad ICE,CRICR,0.400,0.0,0.150,0.060,no,yes
330,Satellite Networks,AeroMobile AS,NORAM,1.120,0.0,0.280,1.400,no,no


In [78]:
# Отличия в тарифах

cols_p4 = ['COUNTRY_NAME','NETWORK_NAME','NETWORK_ID','TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'MOC','MTC','SMS','DATA','is_discounted']

#df2=df.loc[((df.MB_S2.notnull()) | (df.DATA.notnull())) & (df.is_discounted == False), cols_p4].copy()
df2=df.loc[((df.MB_S2.notnull()) | (df.DATA.notnull())) & (df.is_discounted.notnull()), cols_p4].copy()

df_diff = df2[(df2.MB_S2.notnull()) & (\
     ~np.isclose(df2['MOC_S2'], df2['MOC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['MTC_S2'], df2['MTC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['SMS_S2'], df2['SMS'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['MB_S2'], df2['DATA'], atol=0.00001, equal_nan=True)
)]

display(df_diff)

,COUNTRY_NAME,NETWORK_NAME,NETWORK_ID,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,MOC,MTC,SMS,DATA,is_discounted
112,India,Bharti Airtel Limited (Delhi),412.0,IND10,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
113,India,Bharti Airtel Limited (Delhi),412.0,IND11,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
114,India,Bharti Airtel Limited (Delhi),412.0,IND12,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
115,India,Bharti Airtel Limited (Delhi),412.0,IND13,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
116,India,Bharti Airtel Limited (Delhi),412.0,IND14,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
117,India,Bharti Airtel Limited (Delhi),412.0,IND15,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
118,India,Bharti Airtel Limited (Delhi),412.0,IND16,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
119,India,Bharti Airtel Limited (Delhi),412.0,INDA1,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
120,India,Bharti Airtel Limited (Delhi),412.0,INDA2,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True
121,India,Bharti Airtel Limited (Delhi),412.0,INDA3,0.40,0.0,0.140,0.04,0.4100,0.0,0.1400,0.040,True


In [87]:
# Новые тарифы P4 для загрузки в RoamDb

file_partner_roamdb = 'p4_roamdb_result.csv'

print("Новые TADIG P4:")
cols_p4 = ['TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'P4_LTE','Voice_Increment_SEC','Data_Increment_KB']

df_rdb = df.loc[((df.MB_S2.notnull()) & (df.SPONSOR_ID.isnull())
                 & (df.TADIG_CODE.notnull())), cols_p4]

df_rdb['IsDiscounted (Yes/No)'] = 'Yes'
df_rdb.rename({'TADIG_CODE':'TADIG CODE','MOC_S2':'Mobile Origination','MTC_S2':'Mobile Termination','SMS_S2':'SMS','MB_S2':'DATA',
          'Voice_Increment_SEC':'Voice Increment','Data_Increment_KB':'Data Increment','P4_LTE':'Support4G(Yes/No)'},
         axis=1,inplace=True)

df_rdb['Support4G(Yes/No)'].replace({'yes':'Yes','no':'No'},inplace=True)

columns_ordered = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

df_rdb[columns_ordered].to_csv(join(load_folder,file_partner_roamdb),index=False)

df_rdb[columns_ordered]

Новые TADIG P4:


,TADIG CODE,Mobile Origination,Mobile Termination,SMS,DATA,Voice Increment,Data Increment,IsDiscounted (Yes/No),Support4G(Yes/No)
24,BIHPT,0.520,0.0,0.075,0.014,1,1.0,Yes,No
204,MNEPM,0.600,0.0,0.035,0.019,1,1.0,Yes,Yes
205,YUGPM,0.600,0.0,0.035,0.019,1,1.0,Yes,Yes
210,MMRTN,0.600,0.0,0.035,0.019,1,1.0,Yes,No
273,THADT,0.600,0.0,0.035,0.019,1,1.0,Yes,Yes
296,ALBAM,0.449,0.0,0.052,0.040,1,1.0,Yes,Yes
300,BGDGP,0.600,0.0,0.035,0.019,1,1.0,Yes,No
311,CODAC,0.700,0.0,0.140,4.200,1,1.0,Yes,No
312,CRICR,0.400,0.0,0.150,0.060,60,1.0,Yes,No
330,NORAM,1.120,0.0,0.280,1.400,60,1.0,Yes,No
